Site:http://web.stanford.edu/class/cs20si/

Lecture: 
Step 1: Assemble a graph
Step 2: User a session to execute the graph

What is a Tensor?
0-d Tensor - Scalar
1-d Tensor - Vector
2-d Tensor - Matrix
..so on

In [1]:
import tensorflow as tf
import numpy as np

ImportError: No module named tensorflow

In [2]:
a = tf.add(2,3)
print(a)

Tensor("Add:0", shape=(), dtype=int32)


Printing a does not give the result, but it gives the node information. The node has not been executed.

In [3]:
b = tf.add(3.0, 5.0)
print(b)

Tensor("Add_1:0", shape=(), dtype=float32)


Tensors are just data. 
Data Flow is the Tensor flow graph
Now how to get the value of a and b. Run it as part of session. Running in a session means that running with all available resources.

In [4]:
#Create a session; Run the code for 'a' and print the result.
sess = tf.Session()
print(sess.run(a)) # Tensor looks at value of node a and calculates the node value.
sess.close()

5


In [5]:
with tf.Session() as sess:
    print(sess.run(a)) # Using "With"

5


tf.Session(): A Session object encapsulates the environment in which Operation objects are executed and Tensor objects are evaluated.

Session will also allocate memory to store the current values of the variables

### Another Tensor graph

In [6]:
x = 2
y = 3
op1 = tf.add(x,y)
op2 = tf.multiply(x,y)
op3 = tf.pow(op2, op1)
with tf.Session() as sess:
    op3 = sess.run(op3)
    print(op3)

7776


In the above example; just running the node for op3 enumerates all the previous nodes which are required to evaluate op3

In [7]:
x = 2
y = 3
op1 = tf.add(x,y)
op2 = tf.multiply(x,y)

useless = tf.multiply(x, op1)

op3 = tf.pow(op2, op1)
with tf.Session() as sess:
    op3 = sess.run(op3)
    print(op3)

7776


useless is not required in the sess.run for op3; 'useless' is not evaulated. If we need to get output then it will be run.

However, you can explicitly calculate the useless part if needed

In [8]:
x = 2
y = 3
op1 = tf.add(x,y)
op2 = tf.multiply(x,y)

useless = tf.multiply(x, op1)

op3 = tf.pow(op2, op1)
with tf.Session() as sess:
    k = sess.run([op3,useless])
print(k)

[7776, 10]


The computations are divided into graphs and subgraphs and each sub-graph cna be evaluated independently on different kind of machines.

In [9]:
# General Matrix mulitplication
a = tf.constant([2.0, 5.0, 3.0, 4.0], shape = [2,2], name = 'a')
b = tf.constant([2.0, 5.0,9.0, 7.0], shape = [2,2], name = 'b')
c = tf.matmul(a,b)

with tf.Session() as sess: 
    print(sess.run(c))

[[49. 45.]
 [42. 43.]]


In [10]:
# Distributed Computation on specific device
with tf.device('/cpu:0'):
    a = tf.constant([2.0, 5.0, 3.0, 4.0], shape = [2,2], name = 'a')
    b = tf.constant([2.0, 5.0,9.0, 7.0], shape = [2,2], name = 'b')
    c = tf.matmul(a,b)
    
# Create a session with log_device_placement set to true
sess = tf.Session(config = tf.ConfigProto(log_device_placement = True))

# Run the op
print(sess.run(c))

[[49. 45.]
 [42. 43.]]


What about more graph?
Tensorflow provides default graph. You can build more graphs, but recommendation is to use one graph.

### Why Graphs?
Lecture covers a discussion on creating a new graph and adding node to it for computation. General reco from course instructor is to use only 1 graph. 

Why use graphs though ?
- We can partition graphs and run the subgraphs that are relevant for particular computation.
- Partiton the computation logic into smaller pieces which can be then distributed to different gpu/cps
- Directed Acyclyic graph models are used in Neural networks and this architecture aligns with that.

20180422- Updated notes
- Prefer not to create multiple graphs
- multiple graphs require multiple sessions and each session will try to use all the resources.
- Difficult to pass data between graps; apparently requires python/numpy to do that and breaks the distributed paradigm
- preference: Have disconnected subgraphs within one graph

In [11]:
#Create a graph and add computations to it

g = tf.Graph()
with g.as_default():
    x = tf.add(3,5)
sess = tf.Session(graph = g)
with tf.Session() as sess:
    sess.run(x)

ValueError: Fetch argument <tf.Tensor 'Add:0' shape=() dtype=int32> cannot be interpreted as a Tensor. (Tensor Tensor("Add:0", shape=(), dtype=int32) is not an element of this graph.)

In [13]:
#Visualizing the graph with tensor board
import tensorflow as tf

a = tf.constant(2, name = "AAA")
b = tf.constant(3, name = "BBB")
x = tf.add(a,b, name = "ADDITION")

writer = tf.summary.FileWriter('./graphs',tf.get_default_graph())
with tf.Session() as sess:
    #writer = tf.summary.FileWriter('./graphs', sess.graph)
    print(sess.run(x))
writer.close() 
#Note that the graph contains previously defined variables as well.

5


To launch tensorboard do the following:
1. Launch a new python terminal
2. Type : source activate tensorflow . This activates a tensornflow python environemnt
3. type tensorboard --logdir=/home/animo/projects/py_projects/tf/graphs  

note:
1. get the directory right, else it doesnt find the tensor graphs.
2. after running command #3; ctrl+click on the localhost link it generates

In [14]:
import tensorflow as tf
a = tf.constant(2)
b = tf.constant(3)
x = tf.add(a,b)
writer = tf.summary.FileWriter('./graphs',tf.get_default_graph())
writer.close()

### Tensor Constants

In [15]:
#signature of constant

#tf.constant(value, dtype = None, shape = None, name = 'Const', verify_shape = False)

a = tf.constant([2,2], name = "a")
b = tf.constant([[0,1],[2,3]], name = 'b')
x = tf.multiply(a,b, name = "mul")
with tf.Session() as sess:
    print(sess.run(x))

[[0 2]
 [4 6]]


In [16]:
# Verify shape: Provide value and shape; if the value does not match the shape provided:- it will throw false

a = tf.constant([[0,1],[2,3]], shape = [2,2],verify_shape = True)  # Throws an error if shape does not match
b = tf.constant(5,shape = [2,2]) #verify_shape by default = False; hence no error

with tf.Session() as sess:
    print(sess.run(b))

#With verify_shape = False, it just duplicated to match the shape of 3x3

[[5 5]
 [5 5]]


In [17]:
# More Constants

a = tf.constant([2,2], name = "a")
b = tf.constant([[0,1],[2,3]], name = "b")
x = tf.add(a,b, name = "add")
y = tf.multiply(a,b, name  = "mul") # Elementwise multipcation
with tf.Session() as sess:
    x, y = sess.run([x,y])
    print(x,"\n\n", y)

[[2 3]
 [4 5]] 

 [[0 2]
 [4 6]]


## Tensor of Zeros

In [18]:
#tf.zeros(shape, dtype = tf.float32, name = None)
k = tf.zeros([2,3], tf.int32)
with tf.Session() as sess:
    print(sess.run(k))

[[0 0 0]
 [0 0 0]]


## Tensors of zeros, in specific shape of an input vector

In [19]:
input_tensor = [[0,1],[2,3],[4,5]]
k = tf.zeros_like(input_tensor)
with tf.Session() as sess:
    print(sess.run(k))

[[0 0]
 [0 0]
 [0 0]]


## Tensor of ones

In [20]:
#tf.ones(shape, dtype = tf.float32, name = None, optmize = True)
k = tf.ones([2,3], tf.int32)
with tf.Session() as sess:
    print(sess.run(k))

[[1 1 1]
 [1 1 1]]


In [21]:
input_tensor = tf.zeros([5,5], tf.int32)
output_tensor = tf.ones_like(input_tensor) # Tensor has a shape of another tensor.
with tf.Session() as sess:
    print(sess.run(output_tensor))

[[1 1 1 1 1]
 [1 1 1 1 1]
 [1 1 1 1 1]
 [1 1 1 1 1]
 [1 1 1 1 1]]


## Tensor filled with specific vaue

In [22]:
#tf.fill(dims, value, name = None)
u = tf.fill([2,3],8)
with tf.Session() as sess:
    print(sess.run(u))

[[8 8 8]
 [8 8 8]]


## Sequences and ranges support in tensorflow

In [23]:
#tf.linspace(start, stop , num, name = None)

#u = tf.linspace(10,20.0,2) # Having integer values throws error
v = tf.linspace(10.0,20.0,4)
with tf.Session() as sess:
    #print(sess.run(u))
    print(sess.run(v))

[10.       13.333333 16.666666 20.      ]


In [24]:
# both linspace and lin_space work !
#tf.lin_space(start, stop , num, name = None)

v = tf.lin_space(10.0,20.0,4)
with tf.Session() as sess:
    #print(sess.run(u))
    print(sess.run(v))

[10.       13.333333 16.666666 20.      ]


### Range: similar to lin space

In [25]:
#tf.range(start, limit = None, delta = 1, dtype = None, name = None)

In [26]:
u = tf.range(0,12,3)
v = tf.range(0,12, dtype = "float32")
with tf.Session() as sess:
    print(sess.run(u))
    print(sess.run(v))

[0 3 6 9]
[ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11.]


Tensor objects are not iterable. Hence the following code will be in error
for i in tf.range(4): will throw an error

In [27]:
u = tf.range(0,12,3)
u = np.arange(4) # commenting this will throw an error as u in tf.range is not iterable
k = tf.constant(u*2)

with tf.Session() as sess:
    print(sess.run(k))

[0 2 4 6]


### Randomly Generated Constants

In [28]:
#Normal distribution
a = tf.random_normal([1,10], mean = 0, stddev = 1)
with tf.Session() as sess:
    print(sess.run(a))

[[ 0.54177606  1.5844038  -2.0179288   0.09738325 -0.6062167   0.48555997
  -1.2997484  -0.12043292 -0.05926561 -0.21120031]]


In [29]:
# If the numbers generated are 2SD away from mean, that point will be resampled; hence technically all points sampled are
# within 2SD of mean.
a = tf.truncated_normal([1,10], mean = 0, stddev = 1)
with tf.Session() as sess:
    print(sess.run(a))

[[ 0.43054783 -0.8207443  -0.848327    1.461384    0.16109002 -0.57714725
  -1.5205578   0.5007197  -0.91525126  1.8585218 ]]


In [30]:
# Generate data from Uniform distribution.
a = tf.random_uniform([1,10], minval = 0, maxval = 1, dtype = tf.float32, seed = None, name = None)
with tf.Session() as sess:
    print(sess.run(a))

[[0.7109517  0.6677362  0.05672455 0.65347874 0.42180097 0.32127988
  0.02304196 0.9666219  0.47687066 0.66061723]]


In [31]:
# Order of the rows of the elements are changed.
a = tf.random_shuffle([[1,2],[3,4],[5,6]], seed = None, name = None)
with tf.Session() as sess:
    print(sess.run(a))

[[3 4]
 [5 6]
 [1 2]]


In [32]:
# In this case, the row order is preserved but the 'depth' order can be changed. Called shuffling on 1st dimension
a = tf.random_shuffle([[[1,2],[3,4],[5,6]],[[11,22],[33,44],[55,66]]], seed = None, name = None)
with tf.Session() as sess:
    print(sess.run(a))

[[[ 1  2]
  [ 3  4]
  [ 5  6]]

 [[11 22]
  [33 44]
  [55 66]]]


In [33]:
#tf.random_crop
#size = height and width of the cropping; here we say 2 height and 3 width
a = tf.random_crop([[1,2,3,4,5,6,7],[11,22,33,44,55,66,77],[111,222,333,444,555,666,777]], size = [2,2])
with tf.Session() as sess:
    print(sess.run(a))

[[ 2  3]
 [22 33]]


In [34]:
# tf.multinomial
import numpy as np
b = tf.constant(np.random.normal(size = (3,4)))
# we take b as the log of probability value

a = tf.multinomial(b,5) # we take sample value
with tf.Session() as sess:
    print(sess.run(b),"\n")
    print(sess.run(a))
        # The output is 3x4
        # Each row of b contains log probability of the 4 classes.
        # Each element in row of 'a' is then selecting and element from 0-3 classes based on log probability
print("\nOne more try\n")
a = tf.multinomial(b,10) # we take sample value
with tf.Session() as sess:
    print(sess.run(b), "\n")
    print(sess.run(a))

[[ 1.05521416 -1.18448811  0.43350414  0.43117322]
 [-0.54234964 -1.55083294  0.23625308  0.3638158 ]
 [ 1.14821245  0.97391881 -0.12700748  1.8360801 ]] 

[[3 0 0 0 1]
 [2 0 0 3 3]
 [0 0 3 0 3]]

One more try

[[ 1.05521416 -1.18448811  0.43350414  0.43117322]
 [-0.54234964 -1.55083294  0.23625308  0.3638158 ]
 [ 1.14821245  0.97391881 -0.12700748  1.8360801 ]] 

[[1 2 3 0 3 0 0 0 0 1]
 [1 3 2 3 0 3 0 3 3 2]
 [0 0 3 0 3 3 3 1 0 0]]


In [35]:
# tf.random_gamma : Look up gamma distribution.

In [36]:
# setting random seed for the entire graph
tf.set_random_seed(5)

More Operations
1. Element wise: Add, Sub, Multiply, Div, Exp, Log, Greater, Less, equal ..
2. Array Operation: Concat, Slice, split, Constant, Rank, Shape, Shuffle
3. Matrix: MatMul, MatrixInverse, MatrixDeterminant

3. Stateful Operations: Variable, Assign, AssignAdd 
4. Neural N/w: SoftMax, Sigmoid, ReLU, Convolution2D, MaxPool
5. Checkpointing operations: Save, Restore
6. Queue and Sync operations: Enqueue, Dequeue, MutexAcquire, MutexRelease
7. Control flow operations: Merge, Switch, Enter, Leave, NextIteration



Arithmetic operations
1. tf.abs
2. tf.negative
3. tf.reciprocal
4. tf.square
5. tf.round
6. tf.sqrt
7. tf.pow
8. tf.exp

In [37]:
#Operations
a = tf.constant([3,6])
b = tf.constant([2,2])
u = tf.add(a,b)
v = tf.add_n([a,b,b])
w= tf.multiply(a,b)
#tf.matmul(a,b) Error
x = tf.matmul(tf.reshape(a,[1,2]), tf.reshape(b,[2,1]))
y = tf.div(a,b)
z = tf.mod(a,b)

with tf.Session() as sess:
    print(sess.run(u))
    print(sess.run(v))
    print(sess.run(w))
    print(sess.run(x))
    print(sess.run(y))
    print(sess.run(z))

[5 8]
[ 7 10]
[ 6 12]
[[18]]
[1 3]
[1 0]


In [4]:
# Division !

a = tf.constant([2,2], name = "a")
b = tf.constant([[0,1],[2,3]], name = "b")
with tf.Session() as sess:
    print("tf.div:\n ", sess.run(tf.div(b,a)),"\n")
    print("tf.divide:\n ", sess.run(tf.divide(b,a)),"\n")
    print("tf.truediv:\n ", sess.run(tf.truediv(b,a)),"\n")
    print("tf.floordiv:\n ", sess.run(tf.floordiv(b,a)),"\n")
    #print("tf.realdiv:\n ", sess.run(tf.realdiv(b,a)),"\n")  #creates error and they screws up something in system; If I run this cell again, it gives error
    print("tf.truncatediv:\n ", sess.run(tf.truncatediv(b,a)),"\n")
    print("tf.floor_div:\n ", sess.run(tf.floor_div(b,a)),"\n")

tf.div:
  [[0 0]
 [1 1]] 

tf.divide:
  [[0.  0.5]
 [1.  1.5]] 

tf.truediv:
  [[0.  0.5]
 [1.  1.5]] 

tf.floordiv:
  [[0 0]
 [1 1]] 

tf.truncatediv:
  [[0 0]
 [1 1]] 

tf.floor_div:
  [[0 0]
 [1 1]] 



In [5]:
import tensorflow as tf
d = tf.constant(4, name = "d")
e = tf.constant(6, name = "e")
with tf.Session() as sess:
    print(sess.run(tf.div(e,d)))

1


### Tensorflow DataTypes

TensorFlow takes Python native types: boolean, numeric(int, float), string

- Single value will be converted to 0-d tensors(or scalars)
- List of values will be converted to 1-d tensors - vectors
- lists of lists will be converted to 2-d tensors - matrices

In [80]:
t_0 = 19
u = tf.zeros_like(t_0)
v = tf.ones_like(t_0)

with tf.Session() as sess:
    print(sess.run(u))
    print(sess.run(v))

0
1


In [7]:
t_1 = ['apple','peach','banana']
u = tf.zeros_like(t_1)
#v = tf.ones_like(t_1) #throws an error because not clear what concent on ones is for a string

with tf.Session() as sess:
    print(sess.run(u))



[b'' b'' b'']


In [8]:
t_2 = [[True,False,False],[False,False,True],[False,True,False]]
u = tf.zeros_like(t_2)
v = tf.ones_like(t_2)

with tf.Session() as sess:
    print(sess.run(u))
    print(sess.run(v))

[[False False False]
 [False False False]
 [False False False]]
[[ True  True  True]
 [ True  True  True]
 [ True  True  True]]


### Tensorflow datatypes

- tf.float16,32,64
- tf.bfloat16 - 16bit truncated floating ponit
- tf.complex64,128
- tf.int8,16,32,64
- tf.uint8,16,16
- tf.bool
- tf.string
- tf.qint8,16,32 - quantized int
- tf.quint8,16 - quantized unsigned int
- tf.resource - handle to a mutable resource

### TensorFlow and Numpy Data Type

Tensorflow integrates seamlessly with Numpy

In [10]:
print("TF and NumPy Data type equivalence:", tf.int32 == np.int32)

TF and NumPy Data type equivalence: True


In [11]:
tf.ones([2,2], np.float32)

<tf.Tensor 'ones:0' shape=(2, 2) dtype=float32>

For tf.Sessio.run(fetches): if the the requested fetch is a Tensor, output will be a Numpy ndarray

In [12]:
sess = tf.Session()
a = tf.zeros([2,3], np.int32)
print(type(a))
a = sess.run(a)  # In general,avoid rewriting like this, e.g use a_out = sess.run(a)
print(type(a))

<class 'tensorflow.python.framework.ops.Tensor'>
<class 'numpy.ndarray'>


Use TF DataType when possisble, TF has to infer numpy datatypes:

Using Python types to specify TensorFlow objects is quick and easy, and it is useful for prototyping ideas. However, there is an important pitfall in doing it this way. Python types lack the ability to explicitly state the data type, but TensorFlow’s data types are more specific. For example, all integers are the same type, but TensorFlow has 8-bit, 16-bit, 32-bit, and 64-bit integers available. Therefore, if you use a Python type, TensorFlow has to infer which data type you mean. 


It’s possible to convert the data into the appropriate type when you pass it into TensorFlow, but certain data types still may be difficult to declare correctly, such as complex numbers. Because of this, it is recommended to created hand-defined Tensor objects as NumPy arrays.


### Note about constants

In [13]:
my_const  = tf.constant([1.0,2.0], name = "my_const")
with tf.Session() as sess:
    print(sess.graph.as_graph_def())

node {
  name: "a"
  op: "Const"
  attr {
    key: "dtype"
    value {
      type: DT_INT32
    }
  }
  attr {
    key: "value"
    value {
      tensor {
        dtype: DT_INT32
        tensor_shape {
          dim {
            size: 2
          }
        }
        tensor_content: "\002\000\000\000\002\000\000\000"
      }
    }
  }
}
node {
  name: "b"
  op: "Const"
  attr {
    key: "dtype"
    value {
      type: DT_INT32
    }
  }
  attr {
    key: "value"
    value {
      tensor {
        dtype: DT_INT32
        tensor_shape {
          dim {
            size: 2
          }
          dim {
            size: 2
          }
        }
        tensor_content: "\000\000\000\000\001\000\000\000\002\000\000\000\003\000\000\000"
      }
    }
  }
}
node {
  name: "div"
  op: "FloorDiv"
  input: "b"
  input: "a"
  attr {
    key: "T"
    value {
      type: DT_INT32
    }
  }
}
node {
  name: "truediv/Cast"
  op: "Cast"
  input: "b"
  attr {
    key: "DstT"
    value {
      type: DT_DOU

Note that constant definitions are stored in a graph and this makes loading graphs expensive when constants are big

The graph definition is stored in a protobuf (protocol buffers, Google's language-neutral, platform-neutral, extensible mechanism for serializing structured data – think XML, but smaller, faster, and simpler.)

Hence, only use constants for primitive types. Use variables or readers for more data that requires memory.

### Variables

In [18]:
#create variable with tf.Variable

s = tf.Variable(2) #scalar
v = tf.Variable([2,3]) #vector 
m = tf.Variable([[1,2],[3,4]]) #matrix
W = tf.Variable(tf.zeros([784,10])) #784x10 matrix

In [25]:
#create variables with tf.get_variable
s2 = tf.get_variable("scalar_3", initializer = tf.constant(2))
v2 = tf.get_variable("vector_3", initializer = tf.constant([2,3]))
m2 = tf.get_variable("matrix_3", initializer = tf.constant([[1,2],[3,4]]))
W2 = tf.get_variable("big_matrix_3", shape = (784,10), initializer = tf.zeros_initializer())

## Can be only run once; gives error if tried to be reinitialized, if reuse is not set

In [86]:
# Variables support various methods
x = tf.Variable(2)

# x.initializer #initializer op
# x.value() # read operation
# x.assign(..) #Write operation
# x.assign_add(..) #?

In [87]:
# You have always to initialize your variables

#e.g
# init = tf.global_variables_initializer()
# with tf.Session() as sess:
#    sess.run(init)

# Initializing specifc variables:
# init_ab = tf.variables_initializer([a,b])
# with tf.Session() as sess:
#    sess.run(init_ab)

# Initialize a single variable
# W = tf.Variable(tf.zeros([10,10]))
# with tf.Session() as sess:
#    sess.run(W.initializer)

In [27]:
#Eval() a variable

W = tf.Variable(tf.truncated_normal([5,5]))
with tf.Session() as sess:
    sess.run(W.initializer)
    print(W,"\n")
    print(W.eval())   

<tf.Variable 'Variable_9:0' shape=(5, 5) dtype=float32_ref> 

[[ 0.01528348 -0.52057874 -1.7205936  -1.1519074  -0.9202446 ]
 [ 0.6885252   1.449347   -0.23999627  0.70245844 -0.54678637]
 [ 0.03010954 -0.4704021   0.5711186  -0.1621516   0.64585155]
 [ 0.4971588   1.3999316   0.5230011   0.6106415  -0.11909041]
 [ 0.40521947 -0.3328309  -0.94336474 -0.75710934  1.8503329 ]]


In [28]:
#tf.Variable.assing()

W = tf.Variable(10)
W.assign(100)
with tf.Session() as sess:
    sess.run(W.initializer)
    print(W.eval()) # Did not call W.assign(100) in the session hence the value was never changed.

10


In [30]:
#tf.Variable.assign()

W = tf.Variable(10)
assign_op= W.assign(100)
with tf.Session() as sess:
    sess.run(W.initializer)
    sess.run(assign_op)
    print(W.eval()) 

100


In [31]:
# MOre example

my_var = tf.Variable(2)
my_var2 = my_var.assign(2*my_var) # Note that my_var2 is 'assigned' from value at my_var
# i.e this assigned 2*my_var to my_var everytimes my_var2 op is executed.
# Not very clear to me why this happens
with tf.Session() as sess:
    sess.run(my_var.initializer)
    print(sess.run(my_var))
    print(sess.run(my_var))
    print(sess.run(my_var2))
    print(sess.run(my_var2))


2
2
4
8


In [107]:
# Assign_add and #Assign_sub; 
my_var = tf.Variable(10)

with tf.Session() as sess:
    sess.run(my_var.initializer)
    
    #increment by 10
    print(sess.run(my_var.assign_add(10)))
    
    
    #subtract 2
    print(sess.run(my_var.assign_sub(2)))

20
18


In [110]:
# Each session maintains its own copy of the variable

W = tf.Variable(10)

sess1 = tf.Session()
sess2 = tf.Session()

sess1.run(W.initializer)
sess2.run(W.initializer)

print(sess1.run(W.assign_add(10))) # W has different value in Session 1 thans in Session 2
print(sess2.run(W.assign_sub(10)))

sess1.close()
sess2.close()

20
0


In [32]:
# Using a variable to initialze another variable.

W = tf.Variable(tf.truncated_normal([10,10]))
#U = tf.Variable(2 * W) ENsure W is initialized before
U = tf.Variable(2*W.initialized_value())

Control Dependencies
Sometimes, we will have two more two independent ops but you’d like to specify which op should be run first, then you use tf.Graph.control_dependencies(control_inputs)

In [111]:
# Control Dependencies
# Defines which ops should be run first

# with g.control_dependencies([a,b,c]):
    # d and e will only run after a, b, and c have exectued
#    d = 
#    e = 

In [33]:
#Placeholders
#A TF program has 2 phase. 1 phas for gathering the graph nodes and 2nd for using a session to execute operations

a = tf.placeholder(tf.float32, shape = [3])
        #Note shape definition is important, else tensor of any shape will be accepted

b = tf.constant([5,5,5], tf.float32)
c = a + b

#Need to feed the value to a before running
with tf.Session() as sess:
    print(sess.run(c,{a: [1,2,3]})) # tensor name is the key in dict and dictionary values are values that we want to feed


[6. 7. 8.]


In [114]:
# Multiple data points for placeholder:
#Placeholders
#A TF program has 2 phase. 1 phas for gathering the graph nodes and 2nd for using a session to execute operations

a = tf.placeholder(tf.float32, shape = [3])

b = tf.constant([5,5,5], tf.float32)
c = a + b

#Need to feed the value to a before running
with tf.Session() as sess:
    for k in range(5): # Call the image in loop.
        print(sess.run(c,{a: [1,2,k]})) # tensor name is the key in dict and dictionary values are values that we want to feed


[ 6.  7.  5.]
[ 6.  7.  6.]
[ 6.  7.  7.]
[ 6.  7.  8.]
[ 6.  7.  9.]


In [34]:
# feed_dict

a = tf.add(2,5)
b = tf.multiply(a,3)

with tf.Session() as sess:
    replace_dict ={a:15} # new value for a
    print(sess.run(b,feed_dict = replace_dict)) #sess.run can take a second argument as the dicitionary.
    
    

45


In [35]:
#Normal loading:
x = tf.Variable(10)
y = tf.Variable(20)
z = tf.add(x,y)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for _ in range(5):
        print(sess.run(z))

30
30
30
30
30


In [125]:
# Lazy Loading
#Normal loading:
x = tf.Variable(10)
y = tf.Variable(20)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for _ in range(5):
        print(sess.run(tf.add(x,y)))

30
30
30
30
30
